# SETTING ENVIRONMENT


In [46]:
"""# mount the colab with google drive
from google.colab import drive
drive.mount('/content/drive')"""

"# mount the colab with google drive\nfrom google.colab import drive\ndrive.mount('/content/drive')"

In [47]:
# set folder tempat kerja (current working directory)
import os
cwd = "/Users/yusufpradana/Library/CloudStorage/OneDrive-Personal/Pekerjaan BMN/05. 2025/98_monitoring_berita/monitoring-berita"
#cwd = '/content/drive/MyDrive/Monitoring Berita'
os.chdir(cwd)

# MAIN

In [48]:
# Langkah pertama membaca file csv hasil analisis AI sebelumnya
# file terletak di config.json "analisis_ai_output"
# Filter out berita dengan topik_llm "Lainnya"
# Filter out berita dengan importance < 50

import pandas as pd
import json
import logging
from pathlib import Path

# Setup logging untuk error handling
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def load_berita_penting():
    """
    Memuat dan memfilter berita penting dari file hasil analisis AI
    
    Returns:
        pandas.DataFrame: DataFrame berisi berita yang sudah difilter
    """
    try:
        # Baca konfigurasi
        with open('config.json', 'r', encoding='utf-8') as f:
            config = json.load(f)
        
        # Path file analisis AI
        analisis_file = config.get('analisis_ai_output')
        if not analisis_file:
            raise ValueError("analisis_ai_output tidak ditemukan dalam config.json")
        
        # Periksa apakah file ada
        if not Path(analisis_file).exists():
            raise FileNotFoundError(f"File analisis AI tidak ditemukan: {analisis_file}")
        
        # Baca file CSV
        logger.info(f"Membaca file analisis AI: {analisis_file}")
        df = pd.read_csv(analisis_file)
        
        # Filter berita penting
        # 1. Exclude topik_llm "Lainnya"
        # 2. Include importance >= 70
        df_filtered = df[
            (df['topik_llm'] != 'Lainnya') & 
            (df['importance'] >= 70)
        ].copy()
        
        logger.info(f"Total berita: {len(df)}")
        logger.info(f"Berita penting (filtered): {len(df_filtered)}")
        
        if df_filtered.empty:
            logger.warning("Tidak ada berita penting yang memenuhi kriteria!")
            return pd.DataFrame()
        
        # Urutkan berdasarkan importance (descending)
        df_filtered = df_filtered.sort_values('importance', ascending=False)
        
        return df_filtered
        
    except Exception as e:
        logger.error(f"Error dalam load_berita_penting: {str(e)}")
        raise

# Load data berita penting
df_berita_penting = load_berita_penting()
print(f"Berhasil memuat {len(df_berita_penting)} berita penting")
if not df_berita_penting.empty:
    print("\nSample berita penting:")
    print(df_berita_penting[['judul_berita', 'topik_llm', 'importance', 'sentimen']].head())

2025-09-30 15:25:27,922 - INFO - Membaca file analisis AI: 00_hasil_analisis/seluruh_berita/analisis_ai_20250930_deepseek_default.csv
2025-09-30 15:25:27,931 - INFO - Total berita: 225
2025-09-30 15:25:27,931 - INFO - Berita penting (filtered): 107
2025-09-30 15:25:27,931 - INFO - Total berita: 225
2025-09-30 15:25:27,931 - INFO - Berita penting (filtered): 107


Berhasil memuat 107 berita penting

Sample berita penting:
                                          judul_berita topik_llm  importance  \
94   Menkeu Purbaya Sidak ke Kantor Pusat BNI, Ada ...  Kemenkeu        85.0   
118    Cukai Rokok Tak Naik, Penerimaan Turun - KONTAN  Kemenkeu        85.0   
116  Pemerhati Sayangkan Penundaan Kenaikan Cukai R...  Kemenkeu        85.0   
114  Prabowo Perintahkan Bea Cukai Gandeng Ahli Kim...  Kemenkeu        85.0   
113  Saham WIIM, HMSP, GGRM Rontok Usai Menkeu Purb...  Kemenkeu        85.0   

    sentimen  
94   positif  
118  positif  
116  negatif  
114  positif  
113  positif  


In [49]:
# buat format prompt baru untuk menganalisis berita
# tanya ke AI untuk mengetahui
# 1. Resume 
# 2. Dampak ke Kementerian Keuangan (positif, negatif, netral)
# 3. Alasan dampak
# 4. Hal menarik dari berita ini

from openai import OpenAI
import requests
import time
import re
from datetime import datetime

def create_analysis_prompt(judul, artikel, source_domain):
    """
    Membuat prompt untuk analisis berita penting
    
    Args:
        judul (str): Judul berita
        artikel (str): Isi artikel berita
        source_domain (str): Domain sumber berita
    
    Returns:
        str: Prompt untuk AI
    """
    prompt = f"""
Analisis berita berikut dengan detail:

JUDUL: {judul}
SUMBER: {source_domain}
ARTIKEL: {artikel}

Berikan analisis dalam format JSON dengan struktur berikut:
{{
    "resume": "Ringkasan singkat dan jelas dari berita dalam 2-3 kalimat",
    "dampak_kemenkeu": "positif/negatif/netral",
    "alasan_dampak": "Penjelasan detail mengapa berita ini berdampak positif/negatif/netral terhadap Kementerian Keuangan. Jelaskan kaitan dengan kebijakan fiskal, perpajakan, kepabeanan, keuangan negara, atau fungsi lain Kemenkeu",
    "hal_menarik": "Poin-poin menarik atau insights penting dari berita ini yang perlu mendapat perhatian khusus"
}}

Pastikan analisis objektif dan berdasarkan fakta yang ada dalam berita.
Berikan response HANYA dalam format JSON, tanpa teks tambahan.
"""
    return prompt

def parse_ai_response(raw_response):
    """
    Parse response dari AI untuk extract JSON
    
    Args:
        raw_response (str): Raw response dari AI
        
    Returns:
        dict: Parsed JSON atau None jika gagal
    """
    try:
        # Remove markdown code blocks jika ada
        cleaned_response = raw_response.strip()
        
        # Remove ```json dan ``` jika ada
        if cleaned_response.startswith('```json'):
            cleaned_response = cleaned_response[7:]
        if cleaned_response.startswith('```'):
            cleaned_response = cleaned_response[3:]
        if cleaned_response.endswith('```'):
            cleaned_response = cleaned_response[:-3]
        
        cleaned_response = cleaned_response.strip()
        
        # Try parsing as JSON directly
        try:
            return json.loads(cleaned_response)
        except json.JSONDecodeError:
            # Fallback: extract JSON pattern
            json_match = re.search(r'\{.*\}', cleaned_response, re.DOTALL)
            if json_match:
                json_str = json_match.group()
                return json.loads(json_str)
            else:
                raise json.JSONDecodeError("No JSON found", cleaned_response, 0)
                
    except Exception as e:
        logger.error(f"Error parsing AI response: {e}")
        return None

def analyze_with_openai(prompt, api_key, model="gpt-4o-mini"):
    """
    Analisis menggunakan OpenAI API (versi 1.0+)
    
    Args:
        prompt (str): Prompt untuk analisis
        api_key (str): OpenAI API key
        model (str): Model OpenAI yang digunakan
    
    Returns:
        dict: Hasil analisis
    """
    try:
        # Initialize OpenAI client dengan API key
        client = OpenAI(api_key=api_key)
        
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "Anda adalah analis berita ahli yang fokus pada dampak berita terhadap Kementerian Keuangan Indonesia. Selalu berikan response dalam format JSON yang valid."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3,
            max_tokens=1000
        )
        
        raw_content = response.choices[0].message.content
        result = parse_ai_response(raw_content)
        
        if result is None:
            logger.error(f"Failed to parse OpenAI response: {raw_content[:200]}...")
        
        return result
        
    except Exception as e:
        logger.error(f"Error OpenAI analysis: {str(e)}")
        return None

def analyze_with_deepseek(prompt, api_key, base_url="https://api.deepseek.com/v1"):
    """
    Analisis menggunakan DeepSeek API
    
    Args:
        prompt (str): Prompt untuk analisis
        api_key (str): DeepSeek API key
        base_url (str): Base URL DeepSeek API
    
    Returns:
        dict: Hasil analisis
    """
    try:
        headers = {
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json"
        }
        
        data = {
            "model": "deepseek-chat",
            "messages": [
                {"role": "system", "content": "Anda adalah analis berita ahli yang fokus pada dampak berita terhadap Kementerian Keuangan Indonesia. Selalu berikan response dalam format JSON yang valid."},
                {"role": "user", "content": prompt}
            ],
            "temperature": 0.3,
            "max_tokens": 1000
        }
        
        response = requests.post(f"{base_url}/chat/completions", headers=headers, json=data)
        response.raise_for_status()
        
        raw_content = response.json()["choices"][0]["message"]["content"]
        result = parse_ai_response(raw_content)
        
        if result is None:
            logger.error(f"Failed to parse DeepSeek response: {raw_content[:200]}...")
        
        return result
        
    except Exception as e:
        logger.error(f"Error DeepSeek analysis: {str(e)}")
        return None

def analyze_berita_batch(df_berita, ai_provider="openai", api_key=None):
    """
    Analisis batch berita menggunakan AI
    
    Args:
        df_berita (pd.DataFrame): DataFrame berita
        ai_provider (str): Provider AI ("openai" atau "deepseek")
        api_key (str): API key untuk AI provider
    
    Returns:
        pd.DataFrame: DataFrame dengan kolom analisis tambahan
    """
    if api_key is None or api_key.strip() == "":
        logger.warning("API key tidak tersedia, skip analisis AI")
        return df_berita
    
    # Create a copy to avoid modifying original DataFrame
    df_result = df_berita.copy()
    results = []
    
    for i, (idx, row) in enumerate(df_berita.iterrows()):
        try:
            logger.info(f"Menganalisis berita {i+1}/{len(df_berita)}: {row['judul_berita'][:50]}...")
            
            # Buat prompt
            prompt = create_analysis_prompt(
                row['judul_berita'], 
                row['artikel_berita_bersih'], 
                row['source_domain']
            )
            
            # Analisis dengan AI
            if ai_provider == "openai":
                analysis = analyze_with_openai(prompt, api_key)
            elif ai_provider == "deepseek":
                analysis = analyze_with_deepseek(prompt, api_key)
            else:
                raise ValueError(f"AI provider tidak dikenali: {ai_provider}")
            
            if analysis and isinstance(analysis, dict):
                results.append(analysis)
                logger.info(f"✅ Berhasil menganalisis berita {i+1}")
            else:
                # Default jika analisis gagal
                results.append({
                    "resume": "Analisis tidak tersedia",
                    "dampak_kemenkeu": "netral",
                    "alasan_dampak": "Tidak dapat dianalisis",
                    "hal_menarik": "Tidak dapat dianalisis"
                })
                logger.warning(f"⚠️  Analisis gagal untuk berita {i+1}, menggunakan default")
            
            # Delay untuk menghindari rate limiting
            time.sleep(1)
            
        except Exception as e:
            logger.error(f"Error analyzing berita {i+1}: {str(e)}")
            results.append({
                "resume": "Error dalam analisis",
                "dampak_kemenkeu": "netral", 
                "alasan_dampak": f"Error: {str(e)}",
                "hal_menarik": "Tidak dapat dianalisis"
            })
    
    # Tambahkan hasil analisis ke DataFrame
    for i, result in enumerate(results):
        original_idx = df_result.index[i]
        for key, value in result.items():
            df_result.loc[original_idx, f"ai_{key}"] = value
    
    return df_result

print("✅ Fungsi analisis AI telah disiapkan (Updated dengan JSON parsing yang diperbaiki).")
print("Untuk melakukan analisis, gunakan: analyze_berita_batch(df_berita_penting, 'openai', 'YOUR_API_KEY')")
print("Atau: analyze_berita_batch(df_berita_penting, 'deepseek', 'YOUR_API_KEY')")

✅ Fungsi analisis AI telah disiapkan (Updated dengan JSON parsing yang diperbaiki).
Untuk melakukan analisis, gunakan: analyze_berita_batch(df_berita_penting, 'openai', 'YOUR_API_KEY')
Atau: analyze_berita_batch(df_berita_penting, 'deepseek', 'YOUR_API_KEY')


In [50]:
# Setelah mendapat informasi terkait berita penting
# Format cetakan sehingga sesuai dengan format ini:

"""Daftar Berita & Konten [Judul]
Selasa, 30 September 2025 [Tanggal Laporan]
Periode pantauan tanggal 29-30 September 2025 (pukul 14.00 s.d. 06.00 WIB) [Waktu Pemantauan]
	
Media Online [Judul Bagian]
===========


🟢 [Sentimen] Purbaya Yakin Kredit Bank Capai 11 Persen Usai Suntikan Dana Rp200 Triliun : Okezone Economy [Judul Berita]
https://economy.okezone.com/read/2025/09/29/320/3173364/purbaya-yakin-kredit-bank-capai-11-persen-usai-suntikan-dana-rp200-triliun [url]


🟢 [Sentimen] Purbaya Targetkan Ekonomi Indonesia Kuartal IV Tumbuh di Atas 5,5% [Judul Berita]
https://ekbis.sindonews.com/read/1626607/33/purbaya-targetkan-ekonomi-indonesia-kuartal-iv-tumbuh-di-atas-55-1759158548 [url]

🟢 [Sentimen] Indef Sikap Purbaya Soal Cukai Lindungi Pekerja Industri Rokok [Judul Berita]
https://mediaindonesia.com/ekonomi/815843/indef-sikap-purbaya-soal-cukai-lindungi-pekerja-industri-rokok [url]"""

from datetime import datetime, timedelta
import locale

def get_sentiment_emoji(sentimen):
    """
    Mengkonversi sentimen ke emoji
    
    Args:
        sentimen (str): Sentimen berita (positif, negatif, netral)
    
    Returns:
        str: Emoji yang sesuai
    """
    sentiment_map = {
        'positif': '🟢',
        'negatif': '🔴', 
        'netral': '🟡'
    }
    return sentiment_map.get(sentimen.lower(), '🟡')

def format_tanggal_indonesia(date_obj):
    """
    Format tanggal dalam bahasa Indonesia
    
    Args:
        date_obj (datetime): Objek datetime
    
    Returns:
        str: Tanggal dalam format Indonesia
    """
    hari_indo = {
        'Monday': 'Senin',
        'Tuesday': 'Selasa', 
        'Wednesday': 'Rabu',
        'Thursday': 'Kamis',
        'Friday': 'Jumat',
        'Saturday': 'Sabtu',
        'Sunday': 'Minggu'
    }
    
    bulan_indo = {
        'January': 'Januari', 'February': 'Februari', 'March': 'Maret',
        'April': 'April', 'May': 'Mei', 'June': 'Juni',
        'July': 'Juli', 'August': 'Agustus', 'September': 'September',
        'October': 'Oktober', 'November': 'November', 'December': 'Desember'
    }
    
    hari_eng = date_obj.strftime('%A')
    bulan_eng = date_obj.strftime('%B')
    
    hari_id = hari_indo.get(hari_eng, hari_eng)
    bulan_id = bulan_indo.get(bulan_eng, bulan_eng)
    
    return f"{hari_id}, {date_obj.day} {bulan_id} {date_obj.year}"

def generate_daftar_berita_format(df_analyzed, periode_start=None, periode_end=None):
    """
    Generate format daftar berita sesuai template
    
    Args:
        df_analyzed (pd.DataFrame): DataFrame berita yang sudah dianalisis
        periode_start (str): Tanggal mulai periode (YYYY-MM-DD)
        periode_end (str): Tanggal akhir periode (YYYY-MM-DD)
    
    Returns:
        str: Laporan dalam format yang diinginkan
    """
    try:
        if df_analyzed.empty:
            return "Tidak ada berita penting untuk dilaporkan."
        
        # Tanggal laporan (hari ini)
        tanggal_laporan = format_tanggal_indonesia(datetime.now())
        
        # Periode pemantauan
        if periode_start and periode_end:
            start_date = datetime.strptime(periode_start, '%Y-%m-%d')
            end_date = datetime.strptime(periode_end, '%Y-%m-%d')
            periode_text = f"tanggal {start_date.day}-{end_date.day} {format_tanggal_indonesia(end_date).split(', ')[1].split(' ')[1]} {end_date.year}"
        else:
            # Default ke kemarin-hari ini
            hari_ini = datetime.now()
            kemarin = hari_ini - timedelta(days=1)
            periode_text = f"tanggal {kemarin.day}-{hari_ini.day} September 2025"
        
        # Header laporan
        laporan = f"""Daftar Berita & Konten
{tanggal_laporan}
Periode pantauan {periode_text} (pukul 14.00 s.d. 06.00 WIB)

Media Online
===========

"""
        
        # Group berita berdasarkan sentimen untuk urutan yang baik
        df_sorted = df_analyzed.sort_values(['sentimen', 'importance'], ascending=[True, False])
        
        # Generate entry untuk setiap berita
        for idx, row in df_sorted.iterrows():
            emoji = get_sentiment_emoji(row['sentimen'])
            judul_clean = row['judul_berita'].replace('\n', ' ').strip()
            
            # Format entry berita
            berita_entry = f"{emoji} [{row['sentimen'].title()}] {judul_clean}\n{row['url_berita']}\n\n"
            laporan += berita_entry
        
        return laporan
        
    except Exception as e:
        logger.error(f"Error generating daftar berita format: {str(e)}")
        return f"Error dalam generate format: {str(e)}"

def save_daftar_berita(laporan_text, output_dir="00_laporan_cetak"):
    """
    Simpan laporan daftar berita ke file
    
    Args:
        laporan_text (str): Teks laporan
        output_dir (str): Directory output
    
    Returns:
        str: Path file yang disimpan
    """
    try:
        # Buat directory jika belum ada
        Path(output_dir).mkdir(exist_ok=True)
        
        # Nama file dengan timestamp
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"daftar_berita_{timestamp}.txt"
        filepath = Path(output_dir) / filename
        
        # Simpan file
        with open(filepath, 'w', encoding='utf-8') as f:
            f.write(laporan_text)
        
        logger.info(f"Laporan daftar berita disimpan: {filepath}")
        return str(filepath)
        
    except Exception as e:
        logger.error(f"Error saving daftar berita: {str(e)}")
        raise

# Test format dengan data dummy jika ada data
if 'df_berita_penting' in locals() and not df_berita_penting.empty:
    print("=== PREVIEW FORMAT DAFTAR BERITA ===")
    sample_format = generate_daftar_berita_format(df_berita_penting.head(3))
    print(sample_format[:500] + "..." if len(sample_format) > 500 else sample_format)
else:
    print("Fungsi format daftar berita telah disiapkan.")
    print("Gunakan: generate_daftar_berita_format(df_analyzed) untuk generate laporan")

=== PREVIEW FORMAT DAFTAR BERITA ===
Daftar Berita & Konten
Selasa, 30 September 2025
Periode pantauan tanggal 29-30 September 2025 (pukul 14.00 s.d. 06.00 WIB)

Media Online

🔴 [Negatif] Pemerhati Sayangkan Penundaan Kenaikan Cukai Rokok pada 2026 - RRI.co.id
https://rri.co.id/nasional/1866071/pemerhati-sayangkan-penundaan-kenaikan-cukai-rokok-pada-2026

🟢 [Positif] Menkeu Purbaya Sidak ke Kantor Pusat BNI, Ada apa? - Liputan6.com
https://www.liputan6.com/amp/6171242/menkeu-purbaya-sidak-ke-kantor-pusat-bni-ada-apa

🟢 ...


In [51]:
# =============================================================================
# PARALLEL PROCESSING UNTUK MEMPERCEPAT ANALISIS AI
# =============================================================================

import concurrent.futures
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
from functools import partial

def analyze_single_berita(row_data, ai_provider="openai", api_key=None, delay=0.5):
    """
    Analisis single berita untuk parallel processing
    
    Args:
        row_data (tuple): (index, row) dari iterrows()
        ai_provider (str): Provider AI
        api_key (str): API key
        delay (float): Delay antar request (detik)
    
    Returns:
        tuple: (index, result_dict)
    """
    idx, row = row_data
    
    try:
        # Buat prompt
        prompt = create_analysis_prompt(
            row['judul_berita'], 
            row['artikel_berita_bersih'], 
            row['source_domain']
        )
        
        # Analisis dengan AI
        if ai_provider == "openai":
            analysis = analyze_with_openai(prompt, api_key)
        elif ai_provider == "deepseek":
            analysis = analyze_with_deepseek(prompt, api_key)
        else:
            raise ValueError(f"AI provider tidak dikenali: {ai_provider}")
        
        # Delay untuk rate limiting
        if delay > 0:
            time.sleep(delay)
        
        if analysis and isinstance(analysis, dict):
            return (idx, analysis)
        else:
            return (idx, {
                "resume": "Analisis tidak tersedia",
                "dampak_kemenkeu": "netral",
                "alasan_dampak": "Tidak dapat dianalisis",
                "hal_menarik": "Tidak dapat dianalisis"
            })
            
    except Exception as e:
        logger.error(f"Error analyzing berita {idx}: {str(e)}")
        return (idx, {
            "resume": "Error dalam analisis",
            "dampak_kemenkeu": "netral", 
            "alasan_dampak": f"Error: {str(e)}",
            "hal_menarik": "Tidak dapat dianalisis"
        })

def analyze_berita_parallel(df_berita, ai_provider="openai", api_key=None, max_workers=3, delay=0.5):
    """
    Analisis berita menggunakan parallel processing dengan ThreadPoolExecutor
    
    Args:
        df_berita (pd.DataFrame): DataFrame berita
        ai_provider (str): Provider AI ("openai" atau "deepseek")
        api_key (str): API key untuk AI provider
        max_workers (int): Jumlah maksimum thread (default: 3 untuk menghindari rate limit)
        delay (float): Delay antar request dalam detik (default: 0.5)
    
    Returns:
        pd.DataFrame: DataFrame dengan kolom analisis tambahan
    """
    if api_key is None or api_key.strip() == "":
        logger.warning("API key tidak tersedia, skip analisis AI")
        return df_berita
    
    logger.info(f"🚀 Memulai analisis parallel dengan {max_workers} workers...")
    start_time = time.time()
    
    df_result = df_berita.copy()
    
    # Persiapkan data untuk parallel processing
    row_data = list(df_berita.iterrows())
    
    # Fungsi partial untuk menyederhanakan parameter
    analyze_func = partial(
        analyze_single_berita, 
        ai_provider=ai_provider, 
        api_key=api_key, 
        delay=delay
    )
    
    # Parallel processing dengan ThreadPoolExecutor
    results = {}
    completed = 0
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit semua tugas
        future_to_idx = {executor.submit(analyze_func, row): row[0] for row in row_data}
        
        # Collect hasil secara bertahap
        for future in as_completed(future_to_idx):
            try:
                idx, analysis = future.result()
                results[idx] = analysis
                completed += 1
                
                # Progress update
                progress = (completed / len(df_berita)) * 100
                logger.info(f"✅ Progress: {completed}/{len(df_berita)} ({progress:.1f}%)")
                
            except Exception as e:
                original_idx = future_to_idx[future]
                logger.error(f"❌ Error processing berita {original_idx}: {str(e)}")
                results[original_idx] = {
                    "resume": "Error dalam parallel processing",
                    "dampak_kemenkeu": "netral",
                    "alasan_dampak": f"Parallel error: {str(e)}",
                    "hal_menarik": "Tidak dapat dianalisis"
                }
    
    # Tambahkan hasil ke DataFrame
    for idx, result in results.items():
        if isinstance(result, dict):
            for key, value in result.items():
                df_result.loc[idx, f"ai_{key}"] = str(value) if value is not None else ""
    
    elapsed = time.time() - start_time
    logger.info(f"🎉 Analisis parallel selesai dalam {elapsed:.2f} detik")
    logger.info(f"📊 Rata-rata: {elapsed/len(df_berita):.2f} detik per berita")
    
    return df_result

def analyze_berita_batch_optimized(df_berita, ai_provider="openai", api_key=None, batch_size=5, max_workers=3):
    """
    Analisis berita dengan kombinasi batching dan parallel processing
    
    Args:
        df_berita (pd.DataFrame): DataFrame berita
        ai_provider (str): Provider AI
        api_key (str): API key
        batch_size (int): Ukuran batch untuk processing
        max_workers (int): Jumlah thread per batch
    
    Returns:
        pd.DataFrame: DataFrame dengan analisis
    """
    if api_key is None or api_key.strip() == "":
        logger.warning("API key tidak tersedia, skip analisis AI")
        return df_berita
    
    logger.info(f"🔄 Memulai batch processing dengan ukuran batch: {batch_size}")
    
    df_result = df_berita.copy()
    total_batches = (len(df_berita) + batch_size - 1) // batch_size
    
    for batch_num in range(total_batches):
        start_idx = batch_num * batch_size
        end_idx = min((batch_num + 1) * batch_size, len(df_berita))
        
        logger.info(f"📦 Processing batch {batch_num + 1}/{total_batches} (rows {start_idx}-{end_idx-1})")
        
        # Ambil batch data
        batch_df = df_berita.iloc[start_idx:end_idx]
        
        # Analisis batch dengan parallel processing
        batch_result = analyze_berita_parallel(
            batch_df, 
            ai_provider=ai_provider, 
            api_key=api_key, 
            max_workers=max_workers,
            delay=0.3  # Delay lebih kecil karena batch lebih kecil
        )
        
        # Update hasil - perbaiki assignment dengan memastikan compatibility
        for col in batch_result.columns:
            if col.startswith('ai_'):
                # Ensure we're updating existing rows properly
                for idx in batch_result.index:
                    if idx in df_result.index:
                        df_result.loc[idx, col] = batch_result.loc[idx, col]
        
        # Delay antar batch untuk menghindari rate limit
        if batch_num < total_batches - 1:
            logger.info("⏱️  Waiting between batches...")
            time.sleep(2)
    
    return df_result

print("✅ Fungsi parallel processing telah disiapkan!")
print("\n🚀 Optimizations tersedia:")
print("1. analyze_berita_parallel() - Multi-threading dengan ThreadPoolExecutor")
print("2. analyze_berita_batch_optimized() - Kombinasi batching + parallel processing")
print("\n📊 Perkiraan percepatan:")
print("- Sequential: ~1-2 detik per berita")
print("- Parallel (3 workers): ~3-5x lebih cepat")
print("- Batch + Parallel: Optimal untuk dataset besar")

✅ Fungsi parallel processing telah disiapkan!

🚀 Optimizations tersedia:
1. analyze_berita_parallel() - Multi-threading dengan ThreadPoolExecutor
2. analyze_berita_batch_optimized() - Kombinasi batching + parallel processing

📊 Perkiraan percepatan:
- Sequential: ~1-2 detik per berita
- Parallel (3 workers): ~3-5x lebih cepat
- Batch + Parallel: Optimal untuk dataset besar


In [52]:
# Format kedua terkait dengan news update

"""News Update [Judul] 
Menkeu Sidak BNI [Topik yang Dipantau diambil dari config.json "topic_keywords]     
Jakarta, 29 September 2025 (Pukul 19.00 WIB) [Periode Pemantauan]

Pemberitaan mengenai inspeksi mendadak (sidak) ke kantor pusat PT Bank Negara Indonesia (Persero) Tbk atau BNI hari ini tercatat terdapat 33 berita (30 positif dan 3 netral) di media online. 

Sorotan Media Online 
•⁠  ⁠Menkeu melakukan inspeksi mendadak ke kantor pusat BNI untuk melihat bagaimana kerja BNI pada saat rapat direksi berlangsung.  
•⁠  ⁠Kontroversi kenaikan suku bunga deposito valuta asing (valas) dolar AS menjadi 4% yang dilakukan oleh bank-bank Himbara diduga menjadi latar belakang sidak Menkeu tersebut.  
•⁠  ⁠Sebelumnya Menkeu telah menegaskan bahwa isu kenaikan bunga deposito valas bukan instruksinya dan menolak tuduhan bahwa dirinya mendikte kebijakan perbankan. 
•⁠  ⁠Chief Economist Permata Bank, Josua Pardede, menjelaskan risiko yang lebih luas dari kebijakan menaikkan valas dolar AS adalah menguatnya kecenderungan menyimpan kekayaan dalam bentuk dolar. 
•⁠  ⁠Menkeu menyebut kedatangannya hanya untuk mengecek langsung penyaluran kredit dari perbankan, khususnya bank-bank yang menerima penempatan dana negara sebesar Rp200 triliun. 
 
Tautan Media Online: 
 1.⁠ ⁠Purbaya Tiba-tiba Sidak Kantor BNI, Nimbrung Rapat Direksi 
https://www.cnnindonesia.com/ekonomi/20250929134914-532-1278863/purbaya-tiba-tiba-sidak-kantor-bni-nimbrung-rapat-direksi  
 2.⁠ ⁠Mengapa Menteri Purbaya Inspeksi Mendadak BNI? 
https://www.tempo.co/ekonomi/mengapa-menteri-purbaya-inspeksi-mendadak-bni--2074388 
 3.⁠ ⁠Mendadak Sidak ke Kantor BNI, Menkeu Purbaya: Boleh Masuk Enggak Ya 
https://www.beritasatu.com/ekonomi/2926647/mendadak-sidak-ke-kantor-bni-menkeu-purbaya-boleh-masuk-enggak-ya#goog_rewarded 
 4.⁠ ⁠Purbaya Sidak Kantor BNI Saat Direksi Lagi Rapat, Ada Apa? 
https://economy.okezone.com/amp/2025/09/29/320/3173222/purbaya-sidak-kantor-bni-saat-direksi-lagi-rapat-ada-apa 
 5.⁠ ⁠Purbaya Sidak Kantor BNI: Saya Mau Lihat Bagaimana Kerja Mereka 
https://ekbis.sindonews.com/read/1626387/33/purbaya-sidak-kantor-bni-saya-mau-lihat-bagaimana-kerja-mereka-1759129777/5  
 6.⁠ ⁠Menkeu Purbaya Sidak ke Kantor BNI, Ada Apa? 
https://www.idxchannel.com/amp/economics/menkeu-purbaya-sidak-ke-kantor-bni-ada-apa  
"""

def identify_main_topic(df_berita, config_topics):
    """
    Identifikasi topik utama dari berita berdasarkan frequency dan importance
    
    Args:
        df_berita (pd.DataFrame): DataFrame berita
        config_topics (list): Daftar topik keywords dari config
    
    Returns:
        str: Topik utama yang teridentifikasi
    """
    try:
        if df_berita.empty:
            return "Berita Umum"
        
        # Analisis topik berdasarkan subtopik_llm dan importance
        topic_analysis = df_berita.groupby('subtopik_llm').agg({
            'importance': ['mean', 'count'],
            'judul_berita': 'first'
        }).round(2)
        
        # Flatten kolom
        topic_analysis.columns = ['avg_importance', 'count_berita', 'sample_judul']
        
        # Hitung skor gabungan (weighted importance)
        topic_analysis['weighted_score'] = (
            topic_analysis['avg_importance'] * topic_analysis['count_berita']
        )
        
        # Dapatkan topik utama
        main_topic = topic_analysis.sort_values('weighted_score', ascending=False).index[0]
        
        # Cek apakah topik utama ada dalam config topics
        for config_topic in config_topics:
            if config_topic.lower() in main_topic.lower() or main_topic.lower() in config_topic.lower():
                return config_topic.title()
        
        return main_topic.title()
        
    except Exception as e:
        logger.error(f"Error identifying main topic: {str(e)}")
        return "Berita Umum"

def generate_sentiment_summary(df_berita):
    """
    Generate ringkasan sentimen berita
    
    Args:
        df_berita (pd.DataFrame): DataFrame berita
    
    Returns:
        dict: Summary sentimen
    """
    sentiment_count = df_berita['sentimen'].value_counts().to_dict()
    total = len(df_berita)
    
    return {
        'total': total,
        'positif': sentiment_count.get('positif', 0),
        'negatif': sentiment_count.get('negatif', 0), 
        'netral': sentiment_count.get('netral', 0)
    }

def extract_key_points(df_berita, max_points=5):
    """
    Extract poin-poin kunci dari berita untuk sorotan media online
    
    Args:
        df_berita (pd.DataFrame): DataFrame berita
        max_points (int): Maksimal poin yang akan diambil
    
    Returns:
        list: Daftar poin kunci
    """
    try:
        points = []
        
        # Prioritaskan berita dengan importance tinggi
        df_sorted = df_berita.sort_values('importance', ascending=False)
        
        for idx, row in df_sorted.head(max_points).iterrows():
            # Gunakan AI analysis jika ada, kalau tidak gunakan poin_of_interest
            if 'ai_hal_menarik' in row and pd.notna(row['ai_hal_menarik']):
                point = row['ai_hal_menarik']
            elif pd.notna(row['poin_of_interest']):
                point = row['poin_of_interest']
            else:
                # Fallback ke statement pejabat atau excerpt dari artikel
                if pd.notna(row['statement_pejabat']):
                    point = row['statement_pejabat'][:150] + "..."
                else:
                    point = row['artikel_berita_bersih'][:150] + "..."
            
            points.append(point)
        
        return points
        
    except Exception as e:
        logger.error(f"Error extracting key points: {str(e)}")
        return ["Tidak dapat mengekstrak poin kunci dari berita"]

def generate_news_update_format(df_berita, main_topic=None):
    """
    Generate format News Update sesuai template
    
    Args:
        df_berita (pd.DataFrame): DataFrame berita yang sudah dianalisis
        main_topic (str): Topik utama (optional)
    
    Returns:
        str: News Update dalam format yang diinginkan
    """
    try:
        if df_berita.empty:
            return "Tidak ada berita untuk news update."
        
        # Load config untuk topic keywords
        with open('config.json', 'r', encoding='utf-8') as f:
            config = json.load(f)
        
        # Identifikasi topik utama
        if not main_topic:
            main_topic = identify_main_topic(df_berita, config.get('topic_keywords', []))
        
        # Tanggal dan waktu
        tanggal_laporan = format_tanggal_indonesia(datetime.now())
        waktu_laporan = datetime.now().strftime("%H.%M")
        
        # Summary sentimen
        sentiment_summary = generate_sentiment_summary(df_berita)
        
        # Generate sentiment text
        sentiment_text = []
        if sentiment_summary['positif'] > 0:
            sentiment_text.append(f"{sentiment_summary['positif']} positif")
        if sentiment_summary['negatif'] > 0:
            sentiment_text.append(f"{sentiment_summary['negatif']} negatif")
        if sentiment_summary['netral'] > 0:
            sentiment_text.append(f"{sentiment_summary['netral']} netral")
        
        sentiment_string = " dan ".join(sentiment_text) if sentiment_text else "beragam sentimen"
        
        # Extract key points
        key_points = extract_key_points(df_berita)
        
        # Header news update
        news_update = f"""News Update
{main_topic}
Jakarta, {tanggal_laporan} (Pukul {waktu_laporan} WIB)

Pemberitaan mengenai {main_topic.lower()} hari ini tercatat terdapat {sentiment_summary['total']} berita ({sentiment_string}) di media online.

Sorotan Media Online"""
        
        # Tambahkan key points
        for i, point in enumerate(key_points, 1):
            news_update += f"\n• {point}"
        
        news_update += "\n\nTautan Media Online:"
        
        # Tambahkan daftar berita dengan link
        for i, (idx, row) in enumerate(df_berita.head(10).iterrows(), 1):
            judul_clean = row['judul_berita'].replace('\n', ' ').strip()
            news_update += f"\n{i}. {judul_clean}\n{row['url_berita']}"
        
        return news_update
        
    except Exception as e:
        logger.error(f"Error generating news update format: {str(e)}")
        return f"Error dalam generate news update: {str(e)}"

def save_news_update(news_update_text, topic="general", output_dir="00_laporan_cetak"):
    """
    Simpan news update ke file
    
    Args:
        news_update_text (str): Teks news update
        topic (str): Topik untuk nama file
        output_dir (str): Directory output
    
    Returns:
        str: Path file yang disimpan
    """
    try:
        # Buat directory jika belum ada
        Path(output_dir).mkdir(exist_ok=True)
        
        # Nama file dengan timestamp dan topik
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        topic_clean = topic.replace(" ", "_").lower()
        filename = f"news_update_{topic_clean}_{timestamp}.txt"
        filepath = Path(output_dir) / filename
        
        # Simpan file
        with open(filepath, 'w', encoding='utf-8') as f:
            f.write(news_update_text)
        
        logger.info(f"News update disimpan: {filepath}")
        return str(filepath)
        
    except Exception as e:
        logger.error(f"Error saving news update: {str(e)}")
        raise

# Test format dengan data dummy jika ada data
if 'df_berita_penting' in locals() and not df_berita_penting.empty:
    print("=== PREVIEW FORMAT NEWS UPDATE ===")
    sample_news_update = generate_news_update_format(df_berita_penting.head(5))
    print(sample_news_update[:800] + "..." if len(sample_news_update) > 800 else sample_news_update)
else:
    print("Fungsi format news update telah disiapkan.")
    print("Gunakan: generate_news_update_format(df_analyzed) untuk generate news update")

=== PREVIEW FORMAT NEWS UPDATE ===
News Update
Rokok Ilegal
Jakarta, Selasa, 30 September 2025 (Pukul 15.25 WIB)

Pemberitaan mengenai rokok ilegal hari ini tercatat terdapat 5 berita (4 positif dan 1 negatif) di media online.

Sorotan Media Online
• Menteri Keuangan Purbaya Yudhi Sadewa, Direktur Utama BNI Putrama Wahju Setyawan, Wakil Direktur Utama BNI Alexandra Askandar
• Menteri Keuangan Purbaya Yudhi Sadewa
• Menteri Keuangan Purbaya Yudhi Sadewa, Ketua FKBI Tulus Abadi, Pengamat Hananto Wibisono
• Presiden Prabowo Subianto
• Menteri Keuangan Purbaya Yudhi Sadewa, Direktur Komunikasi dan Bimbingan Pengguna Jasa Direktorat Bea Cukai Nirwala Dwi Heryanto

Tautan Media Online:
1. Menkeu Purbaya Sidak ke Kantor Pusat BNI, Ada apa? - Liputan6.com
https://www.liputan6.com/amp/6171242/menkeu-purbaya-sidak-ke-kantor-pusat-bni...


In [ ]:
# Format ketiga 
# Prompt untuk laporan analisis berita

"""Buatlah sebuah dokumen laporan analisis media online dan media sosial dengan struktur dan format sebagai berikut:

1. JUDUL DAN TANGGAL

Judul utama: "Laporan Analisis Media Online dan Media Sosial"
Cantumkan hari, tanggal, dan tahun (contoh: Senin, 29 September 2025)

2. EXECUTIVE SUMMARY

Gunakan pemisah garis seperti =========
Ringkasan harus mencakup poin-poin utama dari pemberitaan media online dan isu-isu terkini, termasuk:
Isu utama (misal: sidak menteri, kebijakan cukai, revisi UU, dll.)
Fokus pemerintah atau kementerian
Pernyataan atau kebijakan penting dari pejabat
3. MEDIA ONLINE

Subjudul: "Media Online"
Topik Berita: Sebutkan topik-topik utama yang dilaporkan [topik diambil dari config.json "topic_keywords"]
Tonasi Berita: Tuliskan sentimen (misal: Netral, Positif, Negatif)
Pesan Kunci dan Analisis:
Bagian ini dibagi menjadi:
ISU KEMENKEU (nomor 1, 2, 3, dst. dengan penjelasan singkat)
ISU NASIONAL DAN INTERNASIONAL (nomor 1, 2, dst. dengan penjelasan singkat)
Kegiatan yang dirujuk: Jelaskan jenis kegiatan (misal: Kegiatan Baru, Tanggal)
Narasumber utama yang dirujuk: Tulis "Belum ada narasumber" jika tidak disebutkan
Daftar Berita: Buat daftar berita dengan format:
Nomor. Judul berita
[URL]

4. FORMAT UMUM

Gunakan pemisah halaman seperti ===== Page X =====
Gunakan tanda tebal untuk judul dan subjudul
Gunakan tanda - untuk poin-poin dalam analisis
Pastikan konsistensi penulisan tanggal, nama, dan istilah"""

def categorize_berita(df_berita):
    """
    Kategorikan berita berdasarkan relevansi dengan Kemenkeu
    
    Args:
        df_berita (pd.DataFrame): DataFrame berita
    
    Returns:
        dict: Dictionary dengan kategori berita
    """
    try:
        # Keywords untuk identifikasi isu Kemenkeu
        kemenkeu_keywords = [
            'kemenkeu', 'kementerian keuangan', 'menteri keuangan', 'menkeu',
            'pajak', 'bea cukai', 'anggaran', 'fiskal', 'apbn', 'apbd',
            'sbn', 'obligasi', 'deficit', 'surplus', 'pembiayaan',
            'penerimaan negara', 'belanja negara', 'purbaya'
        ]
        
        # Kategorisasi
        isu_kemenkeu = []
        isu_nasional_internasional = []
        
        for idx, row in df_berita.iterrows():
            # Gabungkan teks untuk analisis
            full_text = f"{row['judul_berita']} {row['artikel_berita_bersih']}"
            full_text_lower = full_text.lower()
            
            # Cek apakah mengandung keyword Kemenkeu
            is_kemenkeu = any(keyword in full_text_lower for keyword in kemenkeu_keywords)
            
            if is_kemenkeu or row['kategori_isu'] == 'Kemenkeu':
                isu_kemenkeu.append(row)
            else:
                isu_nasional_internasional.append(row)
        
        return {
            'kemenkeu': pd.DataFrame(isu_kemenkeu),
            'nasional_internasional': pd.DataFrame(isu_nasional_internasional)
        }
        
    except Exception as e:
        logger.error(f"Error categorizing berita: {str(e)}")
        return {'kemenkeu': pd.DataFrame(), 'nasional_internasional': df_berita}

def extract_narasumber(df_berita):
    """
    Extract narasumber utama dari berita
    
    Args:
        df_berita (pd.DataFrame): DataFrame berita
    
    Returns:
        list: Daftar narasumber yang teridentifikasi
    """
    narasumber_list = []
    
    for idx, row in df_berita.iterrows():
        if pd.notna(row['poin_of_interest']) and row['poin_of_interest'].strip():
            narasumber_list.append(row['poin_of_interest'])
        elif pd.notna(row['statement_pejabat']) and row['statement_pejabat'].strip():
            # Extract nama dari statement (ambil kata pertama yang kapital)
            words = row['statement_pejabat'].split()
            for word in words[:5]:  # Cek 5 kata pertama
                if word.istitle() and len(word) > 3:
                    narasumber_list.append(word)
                    break
    
    # Hapus duplikasi dan return unique narasumber
    unique_narasumber = list(set(narasumber_list))[:5]  # Ambil max 5
    
    return unique_narasumber if unique_narasumber else ["Belum ada narasumber"]

def generate_executive_summary(df_berita, categorized_berita):
    """
    Generate executive summary untuk laporan
    
    Args:
        df_berita (pd.DataFrame): DataFrame semua berita
        categorized_berita (dict): Berita yang sudah dikategorikan
    
    Returns:
        str: Executive summary text
    """
    try:
        # Identifikasi isu utama
        if not categorized_berita['kemenkeu'].empty:
            main_kemenkeu_issue = categorized_berita['kemenkeu'].iloc[0]['subtopik_llm']
        else:
            main_kemenkeu_issue = "Tidak ada isu Kemenkeu utama"
        
        # Hitung statistik
        total_berita = len(df_berita)
        kemenkeu_count = len(categorized_berita['kemenkeu'])
        nasional_count = len(categorized_berita['nasional_internasional'])
        
        # Sentimen overview
        sentiment_summary = generate_sentiment_summary(df_berita)
        
        # Buat executive summary
        executive_summary = f"""
Periode pemantauan ini mencatat {total_berita} berita penting yang memenuhi kriteria analisis. 
Dari total tersebut, {kemenkeu_count} berita terkait langsung dengan Kementerian Keuangan dan {nasional_count} berita terkait isu nasional/internasional.

Isu utama yang mendominasi pemberitaan Kemenkeu adalah {main_kemenkeu_issue}. 
Secara keseluruhan, tonasi pemberitaan menunjukkan {sentiment_summary['positif']} berita positif, 
{sentiment_summary['negatif']} berita negatif, dan {sentiment_summary['netral']} berita netral.

Fokus pemerintah pada periode ini terkonsentrasi pada implementasi kebijakan fiskal dan 
monitoring pelaksanaan program prioritas nasional.
"""
        
        return executive_summary.strip()
        
    except Exception as e:
        logger.error(f"Error generating executive summary: {str(e)}")
        return "Error dalam membuat executive summary"

def generate_laporan_analisis_lengkap(df_berita, periode_start=None, periode_end=None):
    """
    Generate laporan analisis media online dan media sosial lengkap
    
    Args:
        df_berita (pd.DataFrame): DataFrame berita yang sudah dianalisis
        periode_start (str): Tanggal mulai periode
        periode_end (str): Tanggal akhir periode
    
    Returns:
        str: Laporan lengkap dalam format yang diinginkan
    """
    try:
        if df_berita.empty:
            return "Tidak ada data berita untuk dianalisis."
        
        # Load config
        with open('config.json', 'r', encoding='utf-8') as f:
            config = json.load(f)
        
        # Header laporan
        tanggal_laporan = format_tanggal_indonesia(datetime.now())
        
        laporan = f"""**Laporan Analisis Media Online dan Media Sosial**
{tanggal_laporan}

=========================================================

**EXECUTIVE SUMMARY**
=========================================================
"""
        
        # Kategorisasi berita
        categorized_berita = categorize_berita(df_berita)
        
        # Executive Summary
        exec_summary = generate_executive_summary(df_berita, categorized_berita)
        laporan += exec_summary
        
        laporan += "\n\n=========================================================\n\n"
        
        # Section Media Online
        laporan += "**MEDIA ONLINE**\n\n"
        
        # Topik Berita
        topic_keywords = config.get('topic_keywords', [])
        topik_text = ", ".join(topic_keywords) if topic_keywords else "Beragam topik"
        laporan += f"**Topik Berita:** {topik_text}\n\n"
        
        # Tonasi Berita
        sentiment_summary = generate_sentiment_summary(df_berita)
        if sentiment_summary['positif'] > sentiment_summary['negatif']:
            tonasi_dominan = "Positif"
        elif sentiment_summary['negatif'] > sentiment_summary['positif']:
            tonasi_dominan = "Negatif"
        else:
            tonasi_dominan = "Netral"
        
        laporan += f"**Tonasi Berita:** {tonasi_dominan}\n\n"
        
        # Pesan Kunci dan Analisis
        laporan += "**Pesan Kunci dan Analisis:**\n\n"
        
        # ISU KEMENKEU
        laporan += "**ISU KEMENKEU**\n"
        if not categorized_berita['kemenkeu'].empty:
            for i, (idx, row) in enumerate(categorized_berita['kemenkeu'].head(5).iterrows(), 1):
                if 'ai_resume' in row and pd.notna(row['ai_resume']):
                    desc = row['ai_resume']
                else:
                    desc = row['artikel_berita_bersih'][:200] + "..."
                laporan += f"{i}. {desc}\n"
        else:
            laporan += "1. Tidak ada isu Kemenkeu yang dominan pada periode ini\n"
        
        laporan += "\n"
        
        # ISU NASIONAL DAN INTERNASIONAL
        laporan += "**ISU NASIONAL DAN INTERNASIONAL**\n"
        if not categorized_berita['nasional_internasional'].empty:
            for i, (idx, row) in enumerate(categorized_berita['nasional_internasional'].head(5).iterrows(), 1):
                if 'ai_resume' in row and pd.notna(row['ai_resume']):
                    desc = row['ai_resume']
                else:
                    desc = row['artikel_berita_bersih'][:200] + "..."
                laporan += f"{i}. {desc}\n"
        else:
            laporan += "1. Tidak ada isu nasional/internasional yang signifikan\n"
        
        laporan += "\n"
        
        # Kegiatan yang dirujuk
        laporan += f"**Kegiatan yang dirujuk:** Kegiatan Pemantauan Berita, {tanggal_laporan}\n\n"
        
        # Narasumber utama
        narasumber_list = extract_narasumber(df_berita)
        narasumber_text = ", ".join(narasumber_list[:3]) if len(narasumber_list) > 0 else "Belum ada narasumber"
        laporan += f"**Narasumber utama yang dirujuk:** {narasumber_text}\n\n"
        
        # Daftar Berita
        laporan += "**Daftar Berita:**\n"
        for i, (idx, row) in enumerate(df_berita.iterrows(), 1):
            judul_clean = row['judul_berita'].replace('\n', ' ').strip()
            laporan += f"{i}. {judul_clean}\n[{row['url_berita']}]\n\n"
        
        # Page separator
        laporan += "\n===== Page 1 =====\n"
        
        return laporan
        
    except Exception as e:
        logger.error(f"Error generating laporan analisis lengkap: {str(e)}")
        return f"Error dalam generate laporan: {str(e)}"

def save_laporan_analisis_lengkap(laporan_text, output_dir="00_laporan_cetak"):
    """
    Simpan laporan analisis lengkap ke file
    
    Args:
        laporan_text (str): Teks laporan
        output_dir (str): Directory output
    
    Returns:
        str: Path file yang disimpan
    """
    try:
        # Buat directory jika belum ada
        Path(output_dir).mkdir(exist_ok=True)
        
        # Nama file dengan timestamp
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"laporan_analisis_media_{timestamp}.txt"
        filepath = Path(output_dir) / filename
        
        # Simpan file
        with open(filepath, 'w', encoding='utf-8') as f:
            f.write(laporan_text)
        
        logger.info(f"Laporan analisis lengkap disimpan: {filepath}")
        return str(filepath)
        
    except Exception as e:
        logger.error(f"Error saving laporan analisis: {str(e)}")
        raise

# Test format dengan data dummy jika ada data
if 'df_berita_penting' in locals() and not df_berita_penting.empty:
    print("=== PREVIEW FORMAT LAPORAN ANALISIS LENGKAP ===")
    sample_laporan = generate_laporan_analisis_lengkap(df_berita_penting.head(3))
    print(sample_laporan[:1000] + "..." if len(sample_laporan) > 1000 else sample_laporan)
else:
    print("Fungsi format laporan analisis lengkap telah disiapkan.")
    print("Gunakan: generate_laporan_analisis_lengkap(df_analyzed) untuk generate laporan lengkap")

=== PREVIEW FORMAT LAPORAN ANALISIS LENGKAP ===
**Laporan Analisis Media Online dan Media Sosial**
Selasa, 30 September 2025


**EXECUTIVE SUMMARY**
Periode pemantauan ini mencatat 3 berita penting yang memenuhi kriteria analisis. 
Dari total tersebut, 3 berita terkait langsung dengan Kementerian Keuangan dan 0 berita terkait isu nasional/internasional.

Isu utama yang mendominasi pemberitaan Kemenkeu adalah sidak BNI. 
Secara keseluruhan, tonasi pemberitaan menunjukkan 2 berita positif, 
1 berita negatif, dan 0 berita netral.

Fokus pemerintah pada periode ini terkonsentrasi pada implementasi kebijakan fiskal dan 
monitoring pelaksanaan program prioritas nasional.


**MEDIA ONLINE**

**Topik Berita:** rokok ilegal, makan bergizi gratis, tax amnesty, sidak BNI

**Tonasi Berita:** Positif

**Pesan Kunci dan Analisis:**

**ISU KEMENKEU**
1. Liputan6.com, Jakarta M...


In [54]:
# =============================================================================
# IMPLEMENTASI LENGKAP PIPELINE ANALISIS BERITA PENTING (UPDATED: parallel support)
# =============================================================================

def run_complete_analysis(
    ai_provider="openai",
    api_key=None,
    save_outputs=True,
    processing_method="auto",  # 'auto' | 'sequential' | 'parallel' | 'parallel_safe' | 'parallel_v2' | 'batch'
    limit_articles=None         # batasi jumlah berita untuk uji cepat (None = semua)
):
    """
    Menjalankan pipeline lengkap analisis berita penting dengan dukungan metode pemrosesan fleksibel.

    Args:
        ai_provider (str): Provider AI ("openai" atau "deepseek")
        api_key (str): API key untuk AI provider  
        save_outputs (bool): Simpan output ke file atau tidak
        processing_method (str): Metode analisis ('auto','sequential','parallel','parallel_safe','parallel_v2','batch')
        limit_articles (int|None): Jika diset, hanya analisis N berita pertama (untuk testing)

    Returns:
        dict: Dictionary berisi semua hasil analisis
    """
    print("🚀 Memulai pipeline analisis berita penting...")
    results = {}

    def _choose_method(n):
        # Fallback jika PERFORMANCE_CONFIG belum didefinisikan
        seq_thr = PERFORMANCE_CONFIG.get('sequential_threshold', 10) if 'PERFORMANCE_CONFIG' in globals() else 10
        par_thr = PERFORMANCE_CONFIG.get('parallel_threshold', 50) if 'PERFORMANCE_CONFIG' in globals() else 50
        if n <= seq_thr:
            return 'sequential'
        elif n <= par_thr:
            return 'parallel_v2'  # gunakan versi parallel v2 (stabil + cepat)
        else:
            return 'batch'

    try:
        # 1. Load berita penting
        print("\n📊 Step 1: Loading berita penting...")
        df_berita = load_berita_penting()

        if df_berita.empty:
            print("❌ Tidak ada berita penting yang memenuhi kriteria!")
            return {'error': 'No data'}

        # Optional limit for quick test
        if isinstance(limit_articles, int) and limit_articles > 0:
            df_berita = df_berita.head(limit_articles).copy()
            print(f"⚡ Mode uji cepat: hanya {len(df_berita)} berita pertama dianalisis")

        print(f"✅ Berhasil memuat {len(df_berita)} berita penting")
        results['raw_data'] = df_berita

        # 2. Analisis AI (opsional jika ada API key)
        if api_key:
            print(f"\n🤖 Step 2: Analisis AI ({ai_provider.upper()})...")

            # Tentukan metode
            chosen_method = processing_method
            if processing_method == 'auto':
                chosen_method = _choose_method(len(df_berita))
            print(f"🧠 Metode analisis dipilih: {chosen_method}")

            df_analyzed = df_berita.copy()
            analysis_error = None

            try:
                if chosen_method == 'sequential':
                    print("📝 Menjalankan sequential processing...")
                    df_analyzed = analyze_berita_batch(df_berita, ai_provider, api_key)
                elif chosen_method == 'parallel':
                    print("🧵 Menjalankan parallel processing (ThreadPoolExecutor klasik)...")
                    df_analyzed = analyze_berita_parallel(df_berita, ai_provider=ai_provider, api_key=api_key, max_workers=PERFORMANCE_CONFIG.get('max_workers', 3))
                elif chosen_method == 'parallel_safe':
                    print("🛡️  Menjalankan safe parallel processing...")
                    df_analyzed = analyze_berita_parallel_safe(df_berita, ai_provider=ai_provider, api_key=api_key, max_workers=min(2, PERFORMANCE_CONFIG.get('max_workers', 3)))
                elif chosen_method == 'parallel_v2':
                    print("⚡ Menjalankan parallel_v2 (index reset + assignment aman)...")
                    df_analyzed = analyze_berita_parallel_v2(df_berita, ai_provider=ai_provider, api_key=api_key, max_workers=PERFORMANCE_CONFIG.get('max_workers', 3))
                elif chosen_method == 'batch':
                    print("📦 Menjalankan batch + parallel processing...")
                    df_analyzed = analyze_berita_batch_optimized(
                        df_berita,
                        ai_provider=ai_provider,
                        api_key=api_key,
                        batch_size=PERFORMANCE_CONFIG.get('batch_size', 10),
                        max_workers=PERFORMANCE_CONFIG.get('max_workers', 3)
                    )
                else:
                    print("⚠️  Metode tidak dikenali, fallback ke sequential")
                    df_analyzed = analyze_berita_batch(df_berita, ai_provider, api_key)
            except Exception as e:
                analysis_error = e
                logger.error(f"Error pada metode {chosen_method}: {e}")

            # Fallback jika gagal
            if analysis_error is not None:
                try:
                    print(f"🔁 Fallback ke sequential karena error: {analysis_error}")
                    df_analyzed = analyze_berita_batch(df_berita, ai_provider, api_key)
                except Exception as e2:
                    print(f"❌ Fallback sequential juga gagal: {e2}")
                    logger.error(f"Sequential fallback failed: {e2}")
                    df_analyzed = df_berita  # tanpa kolom AI
            else:
                print("✅ Analisis AI selesai dengan metode:", chosen_method)
        else:
            print("\n⚠️  Step 2: Skip analisis AI (tidak ada API key)")
            df_analyzed = df_berita

        results['analyzed_data'] = df_analyzed

        # 3. Generate Format Daftar Berita
        print("\n📋 Step 3: Generate Daftar Berita...")
        daftar_berita = generate_daftar_berita_format(df_analyzed)
        results['daftar_berita'] = daftar_berita

        if save_outputs:
            daftar_path = save_daftar_berita(daftar_berita)
            results['daftar_berita_file'] = daftar_path
            print(f"✅ Daftar berita disimpan: {daftar_path}")

        # 4. Generate News Update
        print("\n📰 Step 4: Generate News Update...")
        news_update = generate_news_update_format(df_analyzed)
        results['news_update'] = news_update

        if save_outputs:
            news_update_path = save_news_update(news_update)
            results['news_update_file'] = news_update_path
            print(f"✅ News update disimpan: {news_update_path}")

        # 5. Generate Laporan Analisis Lengkap
        print("\n📊 Step 5: Generate Laporan Analisis Lengkap...")
        laporan_lengkap = generate_laporan_analisis_lengkap(df_analyzed)
        results['laporan_lengkap'] = laporan_lengkap

        if save_outputs:
            laporan_path = save_laporan_analisis_lengkap(laporan_lengkap)
            results['laporan_lengkap_file'] = laporan_path
            print(f"✅ Laporan lengkap disimpan: {laporan_path}")

        # 6. Summary hasil
        print(f"\n🎉 Pipeline selesai! Summary:")
        print(f"   - Total berita dianalisis: {len(df_analyzed)}")
        if 'sentimen' in df_analyzed.columns:
            print(f"   - Sentimen positif: {len(df_analyzed[df_analyzed['sentimen'] == 'positif'])}")
            print(f"   - Sentimen negatif: {len(df_analyzed[df_analyzed['sentimen'] == 'negatif'])}")
            print(f"   - Sentimen netral: {len(df_analyzed[df_analyzed['sentimen'] == 'netral'])}")
        ai_cols = [c for c in df_analyzed.columns if c.startswith('ai_')]
        if ai_cols:
            filled = df_analyzed['ai_resume'].notna().sum() if 'ai_resume' in df_analyzed.columns else 'N/A'
            print(f"   - Kolom AI: {ai_cols} (resume terisi: {filled})")

        if save_outputs:
            print(f"\n📁 File output tersimpan di: ./00_laporan_cetak/")

        return results

    except Exception as e:
        logger.error(f"Error dalam pipeline: {str(e)}")
        print(f"❌ Error: {str(e)}")
        return {'error': str(e)}

def quick_preview():
    """
    Quick preview hasil analisis tanpa AI dan tanpa save file
    """
    print("🔍 Quick Preview Mode...")

    try:
        # Load data
        df_berita = load_berita_penting()

        if df_berita.empty:
            print("❌ Tidak ada data berita penting")
            return

        print(f"\n📊 Data Overview:")
        print(f"   Total berita: {len(df_berita)}")
        if 'subtopik_llm' in df_berita.columns:
            print(f"   Topik utama: {df_berita['subtopik_llm'].value_counts().head().to_dict()}")
        if 'sentimen' in df_berita.columns:
            print(f"   Sentimen: {df_berita['sentimen'].value_counts().to_dict()}")
        if 'importance' in df_berita.columns:
            print(f"   Rata-rata importance: {df_berita['importance'].mean():.1f}")

        # Preview format
        print(f"\n📋 Preview Daftar Berita (3 teratas):")
        print("-" * 50)
        preview_daftar = generate_daftar_berita_format(df_berita.head(3))
        print(preview_daftar)

        print(f"\n📰 Preview News Update (3 teratas):")
        print("-" * 50)
        preview_news = generate_news_update_format(df_berita.head(3))
        print(preview_news[:500] + "..." if len(preview_news) > 500 else preview_news)

    except Exception as e:
        print(f"❌ Error: {str(e)}")

# =============================================================================
# CONTOH PENGGUNAAN (UPDATED)
# =============================================================================

print("""
🎯 PIPELINE ANALISIS BERITA PENTING SIAP (Versi Parallel)!

Pilihan penggunaan:

1. QUICK PREVIEW (tanpa AI, tanpa save):
   quick_preview()

2. ANALISIS LENGKAP (dengan AI, otomatis pilih metode):
   results = run_complete_analysis(
       ai_provider="openai", 
       api_key="your-openai-api-key",
       processing_method="auto",  # auto/sequential/parallel/parallel_safe/parallel_v2/batch
       save_outputs=True
   )

3. ANALISIS CEPAT 5 BERITA PERTAMA (uji cepat parallel_v2):
   results = run_complete_analysis(
       ai_provider="openai",
       api_key="your-openai-api-key",
       processing_method="parallel_v2",
       limit_articles=5,
       save_outputs=False
   )

4. ANALISIS TANPA AI (hanya format):
   results = run_complete_analysis(save_outputs=True)

5. FUNGSI INDIVIDUAL:
   - load_berita_penting()
   - generate_daftar_berita_format(df)
   - generate_news_update_format(df)
   - generate_laporan_analisis_lengkap(df)

Gunakan processing_method="parallel_v2" untuk kecepatan + stabilitas, "parallel_safe" jika ingin extra aman.
""")


🎯 PIPELINE ANALISIS BERITA PENTING SIAP (Versi Parallel)!

Pilihan penggunaan:

1. QUICK PREVIEW (tanpa AI, tanpa save):
   quick_preview()

2. ANALISIS LENGKAP (dengan AI, otomatis pilih metode):
   results = run_complete_analysis(
       ai_provider="openai", 
       api_key="your-openai-api-key",
       processing_method="auto",  # auto/sequential/parallel/parallel_safe/parallel_v2/batch
       save_outputs=True
   )

3. ANALISIS CEPAT 5 BERITA PERTAMA (uji cepat parallel_v2):
   results = run_complete_analysis(
       ai_provider="openai",
       api_key="your-openai-api-key",
       processing_method="parallel_v2",
       limit_articles=5,
       save_outputs=False
   )

4. ANALISIS TANPA AI (hanya format):
   results = run_complete_analysis(save_outputs=True)

5. FUNGSI INDIVIDUAL:
   - load_berita_penting()
   - generate_daftar_berita_format(df)
   - generate_news_update_format(df)
   - generate_laporan_analisis_lengkap(df)

Gunakan processing_method="parallel_v2" untuk kecepat

In [55]:
# API Keys - diganti dengan konfigurasi lengkap di cell selanjutnya
DEEPSEEK_API_KEY = os.getenv("DEEPSEEK_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [56]:
# =============================================================================
# KONFIGURASI OPTIMASI DAN PERFORMANCE SETTINGS  
# =============================================================================

# API Keys
DEEPSEEK_API_KEY = os.getenv("DEEPSEEK_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Performance Settings
PERFORMANCE_CONFIG = {
    # Parallel processing settings
    'max_workers': 3,           # Jumlah thread maksimum (sesuaikan dengan rate limit)
    'delay_between_requests': 0.5,  # Delay antar request (detik)
    'batch_size': 10,           # Ukuran batch untuk dataset besar
    'delay_between_batches': 2, # Delay antar batch (detik)
    
    # Threshold untuk memilih metode processing
    'sequential_threshold': 10,  # Jika ≤ 10 berita, gunakan sequential
    'parallel_threshold': 50,    # Jika ≤ 50 berita, gunakan parallel
    # Jika > 50 berita, gunakan batch + parallel
    
    # Timeout dan retry settings  
    'request_timeout': 30,      # Timeout per request (detik)
    'max_retries': 2,          # Maksimum retry per request
}

print("⚙️  Konfigurasi performance telah dimuat!")
print(f"🔧 Settings:")
for key, value in PERFORMANCE_CONFIG.items():
    print(f"   {key}: {value}")

def estimate_processing_time(num_berita, method="auto"):
    """
    Estimasi waktu processing berdasarkan jumlah berita dan metode
    
    Args:
        num_berita (int): Jumlah berita
        method (str): Metode processing ("sequential", "parallel", "batch", "auto")
    
    Returns:
        dict: Estimasi waktu untuk berbagai metode
    """
    # Estimasi waktu per berita (detik)
    time_per_berita = {
        'sequential': 1.5,     # 1.5 detik per berita
        'parallel': 0.5,       # ~3x lebih cepat dengan 3 workers
        'batch': 0.4,          # Sedikit lebih cepat karena batch optimization
    }
    
    estimates = {}
    for method_name, time_per in time_per_berita.items():
        total_time = num_berita * time_per
        estimates[method_name] = {
            'time_seconds': total_time,
            'time_minutes': total_time / 60,
            'time_formatted': f"{total_time//60:.0f}m {total_time%60:.0f}s" if total_time > 60 else f"{total_time:.1f}s"
        }
    
    if method == "auto":
        if num_berita <= PERFORMANCE_CONFIG['sequential_threshold']:
            recommended = 'sequential'
        elif num_berita <= PERFORMANCE_CONFIG['parallel_threshold']:
            recommended = 'parallel'  
        else:
            recommended = 'batch'
        
        estimates['recommended'] = {
            'method': recommended,
            **estimates[recommended]
        }
    
    return estimates

print("\n📊 Fungsi estimasi waktu tersedia: estimate_processing_time(num_berita)")
print("💡 Tips: Jalankan estimate_processing_time(len(df_berita)) sebelum analisis!")

⚙️  Konfigurasi performance telah dimuat!
🔧 Settings:
   max_workers: 3
   delay_between_requests: 0.5
   batch_size: 10
   delay_between_batches: 2
   sequential_threshold: 10
   parallel_threshold: 50
   request_timeout: 30
   max_retries: 2

📊 Fungsi estimasi waktu tersedia: estimate_processing_time(num_berita)
💡 Tips: Jalankan estimate_processing_time(len(df_berita)) sebelum analisis!


In [57]:
# =============================================================================
# PERBAIKAN UNTUK PARALLEL PROCESSING ERRORS
# =============================================================================

def safe_update_dataframe(df_target, df_source, prefix="ai_"):
    """
    Safely update DataFrame dengan hasil dari parallel processing
    
    Args:
        df_target (pd.DataFrame): Target DataFrame
        df_source (pd.DataFrame): Source DataFrame dengan hasil AI
        prefix (str): Prefix untuk kolom baru
    
    Returns:
        pd.DataFrame: Updated DataFrame
    """
    try:
        df_result = df_target.copy()
        
        # Iterate through source DataFrame
        for idx in df_source.index:
            if idx in df_target.index:
                # Get AI columns from source
                ai_cols = [col for col in df_source.columns if col.startswith(prefix)]
                for col in ai_cols:
                    value = df_source.loc[idx, col]
                    # Convert to string to avoid type issues
                    df_result.loc[idx, col] = str(value) if pd.notna(value) else ""
        
        return df_result
        
    except Exception as e:
        logger.error(f"Error in safe_update_dataframe: {str(e)}")
        return df_target

def analyze_berita_parallel_safe(df_berita, ai_provider="openai", api_key=None, max_workers=2, delay=0.5):
    """
    Versi aman dari parallel processing dengan error handling yang lebih baik
    
    Args:
        df_berita (pd.DataFrame): DataFrame berita
        ai_provider (str): Provider AI
        api_key (str): API key
        max_workers (int): Jumlah workers (dikurangi untuk stabilitas)
        delay (float): Delay antar request
    
    Returns:
        pd.DataFrame: DataFrame dengan hasil analisis
    """
    if api_key is None or api_key.strip() == "":
        logger.warning("API key tidak tersedia, skip analisis AI")
        return df_berita
    
    logger.info(f"🚀 Memulai safe parallel analysis dengan {max_workers} workers...")
    start_time = time.time()
    
    df_result = df_berita.copy()
    
    # Initialize AI columns
    ai_columns = ['ai_resume', 'ai_dampak_kemenkeu', 'ai_alasan_dampak', 'ai_hal_menarik']
    for col in ai_columns:
        df_result[col] = ""
    
    # Persiapkan data untuk parallel processing
    row_data = list(df_berita.iterrows())
    
    # Fungsi partial dengan parameter yang dikurangi untuk stabilitas
    analyze_func = partial(
        analyze_single_berita, 
        ai_provider=ai_provider, 
        api_key=api_key, 
        delay=delay
    )
    
    # Parallel processing dengan error handling
    results = {}
    completed = 0
    failed = 0
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit semua tugas
        future_to_idx = {executor.submit(analyze_func, row): row[0] for row in row_data}
        
        # Collect hasil secara bertahap
        for future in as_completed(future_to_idx):
            try:
                idx, analysis = future.result()
                
                if analysis and isinstance(analysis, dict):
                    # Safely assign each AI column
                    for key, value in analysis.items():
                        col_name = f"ai_{key}"
                        if col_name in ai_columns:
                            df_result.loc[idx, col_name] = str(value) if value is not None else ""
                    completed += 1
                else:
                    # Default values for failed analysis
                    for col in ai_columns:
                        df_result.loc[idx, col] = "Analisis tidak tersedia"
                    failed += 1
                
                # Progress update
                total_processed = completed + failed
                progress = (total_processed / len(df_berita)) * 100
                logger.info(f"✅ Progress: {total_processed}/{len(df_berita)} ({progress:.1f}%) - Success: {completed}, Failed: {failed}")
                
            except Exception as e:
                original_idx = future_to_idx[future]
                logger.error(f"❌ Error processing berita {original_idx}: {str(e)}")
                
                # Set default values for error case
                for col in ai_columns:
                    df_result.loc[original_idx, col] = f"Error: {str(e)}"
                failed += 1
    
    elapsed = time.time() - start_time
    logger.info(f"🎉 Safe parallel analysis selesai dalam {elapsed:.2f} detik")
    logger.info(f"📊 Hasil: {completed} berhasil, {failed} gagal dari {len(df_berita)} berita")
    
    return df_result

print("✅ Fungsi parallel processing yang aman telah disiapkan!")
print("📝 Gunakan analyze_berita_parallel_safe() untuk analisis yang lebih stabil")

✅ Fungsi parallel processing yang aman telah disiapkan!
📝 Gunakan analyze_berita_parallel_safe() untuk analisis yang lebih stabil


In [58]:
# =============================================================================
# VERSI PERBAIKAN: analyze_berita_parallel DENGAN NORMALISASI INDEX
# =============================================================================

def analyze_berita_parallel_v2(df_berita, ai_provider="openai", api_key=None, max_workers=3, delay=0.5):
    """
    Versi revisi dari analyze_berita_parallel dengan:
    - Reset index untuk menghindari mismatch
    - Inisialisasi kolom AI terlebih dahulu
    - Assignment langsung menggunakan index baru (0..n-1)
    - Mengembalikan DataFrame dengan index asli (kolom original_index)
    """
    if api_key is None or api_key.strip() == "":
        logger.warning("API key tidak tersedia, skip analisis AI")
        return df_berita

    logger.info(f"🚀 (v2) Memulai analisis parallel dengan {max_workers} workers...")
    start_time = time.time()

    # Simpan index asli dan reset index
    df_work = df_berita.copy()
    df_work = df_work.reset_index().rename(columns={'index': 'original_index'})

    # Inisialisasi kolom AI
    ai_columns = ['ai_resume', 'ai_dampak_kemenkeu', 'ai_alasan_dampak', 'ai_hal_menarik']
    for col in ai_columns:
        if col not in df_work.columns:
            df_work[col] = ""

    # Data untuk parallel (gunakan itertuples agar lebih ringan)
    row_data = list(df_work.itertuples(index=False))  # each is a namedtuple

    def _process_row(nt_row):
        try:
            judul = getattr(nt_row, 'judul_berita')
            artikel = getattr(nt_row, 'artikel_berita_bersih')
            source_domain = getattr(nt_row, 'source_domain')
            idx_local = getattr(nt_row, 'original_index')

            prompt = create_analysis_prompt(judul, artikel, source_domain)
            if ai_provider == "openai":
                analysis = analyze_with_openai(prompt, api_key)
            elif ai_provider == "deepseek":
                analysis = analyze_with_deepseek(prompt, api_key)
            else:
                raise ValueError(f"AI provider tidak dikenali: {ai_provider}")

            if delay > 0:
                time.sleep(delay)

            if analysis and isinstance(analysis, dict):
                return (idx_local, analysis)
            else:
                return (idx_local, {
                    'resume': 'Analisis tidak tersedia',
                    'dampak_kemenkeu': 'netral',
                    'alasan_dampak': 'Tidak dapat dianalisis',
                    'hal_menarik': 'Tidak dapat dianalisis'
                })
        except Exception as e:
            logger.error(f"Error (v2) processing berita {idx_local}: {e}")
            return (idx_local, {
                'resume': 'Error dalam analisis',
                'dampak_kemenkeu': 'netral',
                'alasan_dampak': f'Error: {e}',
                'hal_menarik': 'Tidak dapat dianalisis'
            })

    results = {}
    completed = 0

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_idx = {executor.submit(_process_row, nt_row): getattr(nt_row, 'original_index') for nt_row in row_data}
        for future in as_completed(future_to_idx):
            try:
                idx_orig, analysis = future.result()
                results[idx_orig] = analysis
                completed += 1
                progress = (completed / len(row_data)) * 100
                logger.info(f"✅ (v2) Progress: {completed}/{len(row_data)} ({progress:.1f}%)")
            except Exception as e:
                idx_orig = future_to_idx[future]
                logger.error(f"❌ (v2) Error future idx {idx_orig}: {e}")
                results[idx_orig] = {
                    'resume': 'Error future',
                    'dampak_kemenkeu': 'netral',
                    'alasan_dampak': f'Future error: {e}',
                    'hal_menarik': 'Tidak dapat dianalisis'
                }

    # Map hasil ke df_work berdasarkan original_index
    for idx_orig, analysis in results.items():
        mask = df_work['original_index'] == idx_orig
        for key, value in analysis.items():
            col_name = f"ai_{key}"
            if col_name in ai_columns:
                df_work.loc[mask, col_name] = str(value) if value is not None else ""

    elapsed = time.time() - start_time
    logger.info(f"🎉 (v2) Analisis parallel selesai dalam {elapsed:.2f} detik")

    # Kembalikan ke index asli
    df_work = df_work.set_index('original_index')
    # Pastikan urutan sesuai df_berita awal
    df_work = df_work.loc[df_berita.index]

    return df_work

In [59]:
# Test final: Pipeline lengkap dengan perbaikan error handling + parallel_v2 test
print("🚀 Testing pipeline lengkap (parallel_v2 5 berita)...")

# Estimasi waktu sebelum mulai
if 'df_berita_penting' in locals() and not df_berita_penting.empty:
    time_estimate = estimate_processing_time(len(df_berita_penting))
    print(f"\n⏱️  Estimasi waktu processing:")
    print(f"   Sequential: {time_estimate['sequential']['time_formatted']}")
    print(f"   Parallel: {time_estimate['parallel']['time_formatted']}")
    print(f"   Recommended method: {time_estimate['recommended']['method']} ({time_estimate['recommended']['time_formatted']})")

# Jalankan pipeline lengkap dengan parallel_v2 hanya 5 berita (uji cepat)
try:
    results_fixed = run_complete_analysis(
        ai_provider="openai", 
        api_key=OPENAI_API_KEY,
        processing_method="parallel_v2",
        limit_articles=5,
        save_outputs=False
    )

    # Tampilkan summary hasil
    if 'error' not in results_fixed:
        print("\n🎉 Pipeline (uji cepat) berhasil dijalankan!")
        
        analyzed_df = results_fixed.get('analyzed_data')
        if analyzed_df is not None and not analyzed_df.empty:
            print(f"\n📊 Hasil analisis AI (sample):")
            ai_columns = [col for col in analyzed_df.columns if col.startswith('ai_')]
            if ai_columns:
                print(f"   Kolom AI yang ditambahkan: {ai_columns}")
                
                # Show sample AI analysis
                first_analysis = analyzed_df.iloc[0]
                print(f"\n📝 Sample analisis berita pertama:")
                print(f"   Judul: {first_analysis.get('judul_berita', 'N/A')[:80]}...")
                print(f"   Resume: {first_analysis.get('ai_resume', 'N/A')[:100]}...")
                print(f"   Dampak Kemenkeu: {first_analysis.get('ai_dampak_kemenkeu', 'N/A')}")
                print(f"   Alasan: {first_analysis.get('ai_alasan_dampak', 'N/A')[:100]}...")
                
                # Statistik AI analysis
                non_empty_resume = analyzed_df[analyzed_df['ai_resume'].notna() & (analyzed_df['ai_resume'] != "")].shape[0] if 'ai_resume' in analyzed_df.columns else 0
                print(f"\n📈 Statistik AI Analysis:")
                print(f"   Total berita dianalisis: {len(analyzed_df)}")
                print(f"   Berhasil dianalisis AI: {non_empty_resume}")
                if len(analyzed_df) > 0:
                    print(f"   Success rate: {(non_empty_resume/len(analyzed_df)*100):.1f}%")
            else:
                print("   ⚠️  Tidak ada kolom AI yang ditambahkan")
        
        print(f"\n📁 (Uji cepat) Save outputs dimatikan, tidak ada file yang dibuat.")
        
    else:
        print(f"❌ Pipeline gagal: {results_fixed.get('error')}")
        
except Exception as e:
    print(f"❌ Error dalam testing pipeline: {str(e)}")
    logger.error(f"Pipeline testing error: {str(e)}")
    
    # Fallback: coba quick preview
    print("\n🔄 Mencoba quick preview mode sebagai fallback...")
    try:
        quick_preview()
    except Exception as preview_error:
        print(f"❌ Quick preview juga gagal: {str(preview_error)}")

print("\n✅ Testing selesai!")

2025-09-30 15:25:28,070 - INFO - Membaca file analisis AI: 00_hasil_analisis/seluruh_berita/analisis_ai_20250930_deepseek_default.csv
2025-09-30 15:25:28,076 - INFO - Total berita: 225
2025-09-30 15:25:28,077 - INFO - Berita penting (filtered): 107
2025-09-30 15:25:28,077 - INFO - 🚀 (v2) Memulai analisis parallel dengan 3 workers...
2025-09-30 15:25:28,076 - INFO - Total berita: 225
2025-09-30 15:25:28,077 - INFO - Berita penting (filtered): 107
2025-09-30 15:25:28,077 - INFO - 🚀 (v2) Memulai analisis parallel dengan 3 workers...


🚀 Testing pipeline lengkap (parallel_v2 5 berita)...

⏱️  Estimasi waktu processing:
   Sequential: 2m 40s
   Parallel: 53.5s
   Recommended method: batch (42.8s)
🚀 Memulai pipeline analisis berita penting...

📊 Step 1: Loading berita penting...
⚡ Mode uji cepat: hanya 5 berita pertama dianalisis
✅ Berhasil memuat 5 berita penting

🤖 Step 2: Analisis AI (OPENAI)...
🧠 Metode analisis dipilih: parallel_v2
⚡ Menjalankan parallel_v2 (index reset + assignment aman)...


2025-09-30 15:25:33,545 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-30 15:25:34,054 - INFO - ✅ (v2) Progress: 1/5 (20.0%)
2025-09-30 15:25:34,054 - INFO - ✅ (v2) Progress: 1/5 (20.0%)
2025-09-30 15:25:34,269 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-30 15:25:34,269 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-30 15:25:34,377 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-30 15:25:34,377 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-30 15:25:34,779 - INFO - ✅ (v2) Progress: 2/5 (40.0%)
2025-09-30 15:25:34,779 - INFO - ✅ (v2) Progress: 2/5 (40.0%)
2025-09-30 15:25:34,885 - INFO - ✅ (v2) Progress: 3/5 (60.0%)
2025-09-30 15:25:34,885 - INFO - ✅ (v2) Progress: 3/5 (60.0%)
2025-09-30 15:25:40,486 - INFO - HTTP Request: POST https://api

✅ Analisis AI selesai dengan metode: parallel_v2

📋 Step 3: Generate Daftar Berita...

📰 Step 4: Generate News Update...

📊 Step 5: Generate Laporan Analisis Lengkap...

🎉 Pipeline selesai! Summary:
   - Total berita dianalisis: 5
   - Sentimen positif: 4
   - Sentimen negatif: 1
   - Sentimen netral: 0
   - Kolom AI: ['ai_resume', 'ai_dampak_kemenkeu', 'ai_alasan_dampak', 'ai_hal_menarik'] (resume terisi: 5)

🎉 Pipeline (uji cepat) berhasil dijalankan!

📊 Hasil analisis AI (sample):
   Kolom AI yang ditambahkan: ['ai_resume', 'ai_dampak_kemenkeu', 'ai_alasan_dampak', 'ai_hal_menarik']

📝 Sample analisis berita pertama:
   Judul: Menkeu Purbaya Sidak ke Kantor Pusat BNI, Ada apa? - Liputan6.com...
   Resume: Menteri Keuangan Purbaya Yudhi Sadewa melakukan inspeksi mendadak ke kantor pusat BNI untuk memantau...
   Dampak Kemenkeu: positif
   Alasan: Berita ini berdampak positif terhadap Kementerian Keuangan karena menunjukkan komitmen Menkeu dalam ...

📈 Statistik AI Analysis:
   Total 

In [60]:
# =============================================================================
# EKSEKUSI OTOMATIS SELURUH PIPELINE (FULL RUN)
# =============================================================================
"""
Cara pakai:
1. Pastikan semua cell sebelumnya (fungsi & konfigurasi) sudah dijalankan.
2. Set environment variable API key (export OPENAI_API_KEY=... atau DEEPSEEK_API_KEY=...).
3. Jalankan cell ini untuk memproses semua berita penting dan menghasilkan:
   - Daftar berita (txt)
   - News update (txt)
   - Laporan analisis lengkap (txt)
   - DataFrame hasil analisis AI (csv tambahan) jika AI dijalankan

Parameter yang bisa diatur di bawah:
- FORCE_METHOD: paksa metode tertentu ("auto","sequential","parallel_v2","parallel_safe","parallel","batch")
- LIMIT_ARTICLES: batasi jumlah berita untuk uji cepat (None = semua)
- SAVE_AI_ENRICHED_CSV: simpan DataFrame hasil analisis AI yang diperkaya
"""

import os, time, json
from pathlib import Path
from datetime import datetime

# ================== PARAMETER EKSEKUSI ==================
FORCE_METHOD = "auto"          # ganti ke "parallel_v2" jika ingin paksa
LIMIT_ARTICLES = None           # contoh: 5 untuk uji cepat
SAVE_AI_ENRICHED_CSV = True     # simpan hasil enriched df
OUTPUT_ENRICH_DIR = Path("00_hasil_analisis/berita_penting")
# ========================================================

start_global = time.time()
print("🚀 MENJALANKAN FULL PIPELINE ANALISIS BERITA PENTING")
print(f"⚙️  Metode: {FORCE_METHOD} | Limit: {LIMIT_ARTICLES if LIMIT_ARTICLES else 'Semua'}")

# Deteksi API key dan provider
openai_key = os.getenv("OPENAI_API_KEY")
deepseek_key = os.getenv("DEEPSEEK_API_KEY")

selected_provider = None
selected_key = None
if openai_key and openai_key.strip():
    selected_provider = "openai"
    selected_key = openai_key
elif deepseek_key and deepseek_key.strip():
    selected_provider = "deepseek"
    selected_key = deepseek_key
else:
    print("⚠️  Tidak ada API key ditemukan. Analisis AI akan dilewati.")

# Estimasi waktu (jika data sudah ada di memori)
try:
    if 'df_berita_penting' in globals() and not df_berita_penting.empty:
        est = estimate_processing_time(len(df_berita_penting))
        rec = est.get('recommended', {})
        print("⏱️  Estimasi waktu (berdasarkan jumlah berita saat ini):")
        print(f"   Sequential : {est['sequential']['time_formatted']}")
        print(f"   Parallel   : {est['parallel']['time_formatted']}")
        print(f"   Batch      : {est['batch']['time_formatted']}")
        if rec:
            print(f"   Rekomendasi: {rec['method']} ({rec['time_formatted']})")
except Exception as e:
    print(f"(Info) Gagal menghitung estimasi: {e}")

# Jalankan pipeline
results_full = run_complete_analysis(
    ai_provider=selected_provider if selected_provider else "openai",
    api_key=selected_key,
    processing_method=FORCE_METHOD,
    limit_articles=LIMIT_ARTICLES,
    save_outputs=True
)

# Handling error
if 'error' in results_full:
    print(f"❌ Pipeline gagal: {results_full['error']}")
else:
    print("\n🎯 FULL PIPELINE SELESAI")
    analyzed_df = results_full.get('analyzed_data')
    if analyzed_df is not None and not analyzed_df.empty:
        ai_cols = [c for c in analyzed_df.columns if c.startswith('ai_')]
        print(f"📊 Total berita dianalisis: {len(analyzed_df)} | Kolom AI: {len(ai_cols)}")
        if ai_cols:
            filled = analyzed_df[ai_cols[0]].notna().sum()
            print(f"   Contoh kolom AI: {ai_cols[:4]}")
            print(f"   Isi kolom pertama terisi: {filled} baris")
        # Simpan enriched CSV
        if SAVE_AI_ENRICHED_CSV:
            try:
                OUTPUT_ENRICH_DIR.mkdir(parents=True, exist_ok=True)
                ts = datetime.now().strftime('%Y%m%d_%H%M%S')
                out_csv = OUTPUT_ENRICH_DIR / f"analisis_berita_penting_enriched_{selected_provider or 'noai'}_{ts}.csv"
                analyzed_df.to_csv(out_csv, index=False)
                print(f"💾 DataFrame enriched disimpan: {out_csv}")
            except Exception as e:
                print(f"⚠️  Gagal menyimpan enriched CSV: {e}")
    else:
        print("⚠️  Tidak ada DataFrame hasil analisis yang dapat ditampilkan.")

    # Tampilkan file output yang dihasilkan
    output_files = {k:v for k,v in results_full.items() if k.endswith('_file')}
    if output_files:
        print("\n📁 File teks yang dihasilkan:")
        for k,v in output_files.items():
            print(f"   - {k}: {v}")

elapsed_global = time.time() - start_global
print(f"\n⏲️  Total waktu eksekusi: {elapsed_global:.2f} detik")
print("✅ Selesai.")

2025-09-30 15:25:43,364 - INFO - Membaca file analisis AI: 00_hasil_analisis/seluruh_berita/analisis_ai_20250930_deepseek_default.csv
2025-09-30 15:25:43,374 - INFO - Total berita: 225
2025-09-30 15:25:43,374 - INFO - Total berita: 225
2025-09-30 15:25:43,374 - INFO - Berita penting (filtered): 107
2025-09-30 15:25:43,375 - INFO - 🔄 Memulai batch processing dengan ukuran batch: 10
2025-09-30 15:25:43,375 - INFO - 📦 Processing batch 1/11 (rows 0-9)
2025-09-30 15:25:43,376 - INFO - 🚀 Memulai analisis parallel dengan 3 workers...
2025-09-30 15:25:43,374 - INFO - Berita penting (filtered): 107
2025-09-30 15:25:43,375 - INFO - 🔄 Memulai batch processing dengan ukuran batch: 10
2025-09-30 15:25:43,375 - INFO - 📦 Processing batch 1/11 (rows 0-9)
2025-09-30 15:25:43,376 - INFO - 🚀 Memulai analisis parallel dengan 3 workers...


🚀 MENJALANKAN FULL PIPELINE ANALISIS BERITA PENTING
⚙️  Metode: auto | Limit: Semua
⏱️  Estimasi waktu (berdasarkan jumlah berita saat ini):
   Sequential : 2m 40s
   Parallel   : 53.5s
   Batch      : 42.8s
   Rekomendasi: batch (42.8s)
🚀 Memulai pipeline analisis berita penting...

📊 Step 1: Loading berita penting...
✅ Berhasil memuat 107 berita penting

🤖 Step 2: Analisis AI (OPENAI)...
🧠 Metode analisis dipilih: batch
📦 Menjalankan batch + parallel processing...


2025-09-30 15:25:48,944 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-30 15:25:49,255 - INFO - ✅ Progress: 1/10 (10.0%)
2025-09-30 15:25:49,255 - INFO - ✅ Progress: 1/10 (10.0%)
2025-09-30 15:25:51,066 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-30 15:25:51,066 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-30 15:25:51,375 - INFO - ✅ Progress: 2/10 (20.0%)
2025-09-30 15:25:51,375 - INFO - ✅ Progress: 2/10 (20.0%)
2025-09-30 15:25:51,604 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-30 15:25:51,604 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-30 15:25:51,910 - INFO - ✅ Progress: 3/10 (30.0%)
2025-09-30 15:25:51,910 - INFO - ✅ Progress: 3/10 (30.0%)
2025-09-30 15:25:54,686 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/comp

✅ Analisis AI selesai dengan metode: batch

📋 Step 3: Generate Daftar Berita...
✅ Daftar berita disimpan: 00_laporan_cetak/daftar_berita_20250930_153059.txt

📰 Step 4: Generate News Update...
✅ News update disimpan: 00_laporan_cetak/news_update_general_20250930_153059.txt

📊 Step 5: Generate Laporan Analisis Lengkap...
✅ Laporan lengkap disimpan: 00_laporan_cetak/laporan_analisis_media_20250930_153059.txt

🎉 Pipeline selesai! Summary:
   - Total berita dianalisis: 107
   - Sentimen positif: 78
   - Sentimen negatif: 5
   - Sentimen netral: 24
   - Kolom AI: ['ai_resume', 'ai_dampak_kemenkeu', 'ai_alasan_dampak', 'ai_hal_menarik'] (resume terisi: 107)

📁 File output tersimpan di: ./00_laporan_cetak/

🎯 FULL PIPELINE SELESAI
📊 Total berita dianalisis: 107 | Kolom AI: 4
   Contoh kolom AI: ['ai_resume', 'ai_dampak_kemenkeu', 'ai_alasan_dampak', 'ai_hal_menarik']
   Isi kolom pertama terisi: 107 baris
💾 DataFrame enriched disimpan: 00_hasil_analisis/berita_penting/analisis_berita_penting_e